In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
cd gdrive/My Drive/IEMOCAP-Emotion-Detection-master/code/python_files (1)

/content/gdrive/My Drive/IEMOCAP-Emotion-Detection-master/code/python_files (1)


In [4]:
import numpy as np
import os
import sys
import pandas as pd

import wave
import copy
import math

import keras
from keras.models import Sequential, Model
from keras.layers.core import Dense, Activation
from keras.layers import LSTM, Input, Flatten,Dropout,GlobalAveragePooling2D
from keras.layers.convolutional import Conv2D
from scipy import signal


from helper import *


Using TensorFlow backend.


In [0]:

code_path = os.path.dirname(os.path.realpath(os.getcwd()))
emotions_used = np.array(['ang', 'exc', 'neu', 'sad'])
data_path = code_path + "/../data/sessions/"
sessions = ['Session1', 'Session2', 'Session3', 'Session4', 'Session5']
framerate = 16000

In [0]:
import pickle
with open(data_path + '/../'+'data_collected.pickle', 'rb') as handle:
    data2 = pickle.load(handle)

In [7]:
X_train = []
X_test = []
y_train = []
y_test = []
fs = 16e3
counter = 0
counter1 = 0
for ses_mod in data2:
    if 'impro' in ses_mod['id'] and ses_mod['emotion'] in emotions_used:
        f, t, Sxx = signal.spectrogram(ses_mod['signal'], fs, nperseg=400,noverlap=200)
        Sxx = pad_sequence_into_array(Sxx,maxlen=300,value=0)
        #Sxx[Sxx[:] == 0] = 'nan'
        
        if ses_mod['id'][:5]=="Ses05":
            counter+=1
            X_test.append(Sxx)
            y_test.append(ses_mod['emotion'])
        else:
            counter1+=1
            X_train.append(Sxx)
            y_train.append(ses_mod['emotion'])
        
print(counter)
print(counter1)

X_test = np.array(X_test)
X_train = np.array(X_train)
print(X_train.shape)
print(X_test.shape)

625
2034
(2034, 201, 300)
(625, 201, 300)


In [ ]:
def lstm_model():
    model = Sequential()
    model.add(LSTM(256, return_sequences=True, input_shape=(201, 300)))
    model.add(LSTM(256, return_sequences=False))
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dense(4))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

In [ ]:
model = lstm_model()
model.summary()

In [ ]:
hist = model.fit(X_train,y_train, 
                 batch_size=32, nb_epoch=40, verbose=1, shuffle = True, 
                 validation_data=(X_test,y_test))

In [0]:
X_train = X_train.reshape(-1,201,300,1)
X_test = X_test.reshape(-1,201,300,1)

In [0]:
y  = pd.get_dummies(y_train+y_test)
y_train = y[0:len(y_train)]
y_test = y[len(y_train):]
y_train = np.array(y_train)
y_test = np.array(y_test)

In [0]:
print(data2[0]['signal'].size)
signal1 = data2[0]['signal']
f, t, Sxx = signal.spectrogram(signal1, 16000, nperseg=400)
Sxx.shape

31130


(201, 88)

In [0]:
def make_model():
    in_layer = Input(shape=(201, 300,1))
    x = Conv2D(32,(5,5), activation = 'elu')(in_layer)      
    x = Dropout(0.5)(x)
    x = Conv2D(64,(5,5), activation = 'elu')(x)         
    x = Dropout(0.5)(x)
    x = Conv2D(128, (5,5))(x)                           
    x = GlobalAveragePooling2D()(x)                     
    x = Dense(64,activation='relu')(x)
    x = Dropout(0.5)(x)
    output_layer = Dense(4, activation = "softmax")(x) # softmax output
    model = Model(inputs = in_layer, outputs=output_layer)
    return model

In [0]:
model  =  make_model()
model.compile(optimizer = "adam", loss = "categorical_crossentropy",
                    metrics=["accuracy"])
hist = model.fit(X_train, y_train, 
                 batch_size=32, validation_data=(X_test,y_test),nb_epoch=100, verbose=1, shuffle = True, 
                )

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """


Train on 2034 samples, validate on 625 samples
Epoch 1/100
2034/2034 [==============================] - 79s 39ms/step - loss: 3.8878 - acc: 0.3830 - val_loss: 1.2382 - val_acc: 0.4592
Epoch 2/100
2034/2034 [==============================] - 79s 39ms/step - loss: 1.3491 - acc: 0.4444 - val_loss: 1.2118 - val_acc: 0.4672
Epoch 3/100
2034/2034 [==============================] - 79s 39ms/step - loss: 1.1511 - acc: 0.4990 - val_loss: 1.0873 - val_acc: 0.5152
Epoch 4/100
2034/2034 [==============================] - 79s 39ms/step - loss: 1.1287 - acc: 0.4951 - val_loss: 1.0883 - val_acc: 0.5280
Epoch 5/100
2034/2034 [==============================] - 79s 39ms/step - loss: 1.1074 - acc: 0.4971 - val_loss: 1.0580 - val_acc: 0.5424
Epoch 6/100
2034/2034 [==============================] - 79s 39ms/step - loss: 1.0755 - acc: 0.5138 - val_loss: 1.0520 - val_acc: 0.5696
Epoch 7/100
2034/2034 [==============================] - 79s 39ms/step - loss: 1.0838 - acc: 0.5108 - val_loss: 1.0536 - val_acc: 0

In [0]:
def make_model():
    in_layer = Input(shape=(201, 300,1))
    x = Conv2D(16,(5,5), activation = 'elu')(in_layer)   
    x = Dropout(0.5)(x)
    x = Conv2D(32,(5,5), activation = 'elu')(x)         
    x = Dropout(0.5)(x)
    x = Conv2D(64, (5,5))(x)                           
    x = GlobalAveragePooling2D()(x)                     
    x = Dense(64,activation='relu')(x)
    x = Dropout(0.5)(x)
    output_layer = Dense(4, activation = "softmax")(x) 
    model = Model(inputs = in_layer, outputs=output_layer)
    return model

In [0]:
model  =  make_model()
model.compile(optimizer = "adam", loss = "categorical_crossentropy",
                    metrics=["accuracy"])
hist = model.fit(X_train, y_train, 
                 batch_size=32, validation_data=(X_test,y_test),nb_epoch=100, verbose=1, shuffle = True, 
                )

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """


Train on 2034 samples, validate on 625 samples
Epoch 1/100
2034/2034 [==============================] - 35s 17ms/step - loss: 5.0466 - acc: 0.4086 - val_loss: 1.8293 - val_acc: 0.2176
Epoch 2/100
2034/2034 [==============================] - 30s 15ms/step - loss: 1.6041 - acc: 0.4149 - val_loss: 1.1262 - val_acc: 0.5040
Epoch 3/100
2034/2034 [==============================] - 30s 15ms/step - loss: 1.2910 - acc: 0.4543 - val_loss: 1.0889 - val_acc: 0.5264
Epoch 4/100
2034/2034 [==============================] - 30s 15ms/step - loss: 1.1510 - acc: 0.4877 - val_loss: 1.0663 - val_acc: 0.5584
Epoch 5/100
2034/2034 [==============================] - 30s 15ms/step - loss: 1.1289 - acc: 0.5010 - val_loss: 1.0708 - val_acc: 0.5648
Epoch 6/100
2034/2034 [==============================] - 30s 15ms/step - loss: 1.1168 - acc: 0.5084 - val_loss: 1.0649 - val_acc: 0.5184
Epoch 7/100
2034/2034 [==============================] - 30s 15ms/step - loss: 1.0890 - acc: 0.5177 - val_loss: 1.0439 - val_acc: 0

In [0]:
def make_model():
    in_layer = Input(shape=(201, 300,1))
    x = Conv2D(32,(5,5), activation = 'elu')(in_layer)  
    x = Dropout(0.5)(x)
    x = Conv2D(64,(5,5), activation = 'elu')(x)         
    x = Dropout(0.5)(x)
    x = Conv2D(64,(5,5), activation = 'elu')(x)         
    x = Dropout(0.5)(x)
    x = Conv2D(128, (5,5))(x)                           
    x = GlobalAveragePooling2D()(x)                     
    x = Dense(64,activation='elu')(x)
    x = Dropout(0.5)(x)
    output_layer = Dense(4, activation = "softmax")(x) 
    model = Model(inputs = in_layer, outputs=output_layer)
    return model

In [11]:
model  =  make_model()
model.compile(optimizer = "adam", loss = "categorical_crossentropy",
                    metrics=["accuracy"])
hist = model.fit(X_train, y_train, 
                 batch_size=32, validation_data=(X_test,y_test),nb_epoch=100, verbose=1, shuffle = True, 
                )

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """


Train on 2034 samples, validate on 625 samples
Epoch 1/100
2034/2034 [==============================] - 123s 61ms/step - loss: 5.4109 - acc: 0.3677 - val_loss: 1.3736 - val_acc: 0.3456
Epoch 2/100
2034/2034 [==============================] - 105s 52ms/step - loss: 1.6744 - acc: 0.4145 - val_loss: 1.1766 - val_acc: 0.5024
Epoch 3/100
2034/2034 [==============================] - 105s 52ms/step - loss: 1.2464 - acc: 0.4808 - val_loss: 1.0742 - val_acc: 0.5376
Epoch 4/100
2034/2034 [==============================] - 105s 52ms/step - loss: 1.1574 - acc: 0.5059 - val_loss: 1.0900 - val_acc: 0.5296
Epoch 5/100
2034/2034 [==============================] - 105s 52ms/step - loss: 1.2080 - acc: 0.4690 - val_loss: 1.0548 - val_acc: 0.5248
Epoch 6/100
2034/2034 [==============================] - 105s 52ms/step - loss: 1.1766 - acc: 0.4931 - val_loss: 1.0678 - val_acc: 0.5648
Epoch 7/100
2034/2034 [==============================] - 105s 52ms/step - loss: 1.1621 - acc: 0.5034 - val_loss: 1.0970 - val